# SAR on MovieLens

Simple Algorithm for Recommendation (SAR) is a fast and scalable algorithm for personalized recommendations based on user transaction history. It produces easily explainable and interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

*NOTE: This notebook is based on code from [Recommenders library](https://github.com/recommenders-team/recommenders), under MIT license.*

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users that interacted with one item are also likely to have interacted with the other. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matrix with an affinity vector

### Notes to use SAR properly:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

## 0 Global Settings and Imports

In [1]:
import logging
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.python_utils import binarize
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.sar import SAR
from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)

## 1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>,<Time>,[<Event Type>], [<Event Weight>]`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. Each event type can be assigned a different weight, for example, we might assign a “buy” event a weight of 10, while a “view” event might only have a weight of 1.

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [2]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

# Other data settings
USER_COL = "userID"
ITEM_COL = "itemID"
RATING_COL = "rating"
TIMESTAMP_COL = "timestamp"
PREDICTION_COL = "prediction"

# Train test split ratio
SPLIT_RATIO = 0.75

# Model settings
SIMILARITY_TYPE = "jaccard"
TIME_DECAY = 30 # number of days until the weight of the ratings are decayed by 1/2

SEED = 42

logging.basicConfig(level=logging.DEBUG, format="%(levelname)-8s %(message)s")

### 1.1 Download and use the MovieLens Dataset

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data[RATING_COL] = data[RATING_COL].astype(np.float32)

data.head()

DEBUG    Starting new HTTPS connection (1): files.grouplens.org:443
DEBUG    https://files.grouplens.org:443 "GET /datasets/movielens/ml-100k.zip HTTP/1.1" 200 4924029
INFO     Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████| 4.81k/4.81k [00:01<00:00, 4.19kKB/s]


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### 1.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.

In [4]:
train, test = python_stratified_split(data, ratio=SPLIT_RATIO, filter_by="user", col_user=USER_COL, col_item=ITEM_COL, seed=SEED)

In [5]:
print("""
Train:
Total Ratings: {train_total}
Unique Users: {train_users}
Unique Items: {train_items}

Test:
Total Ratings: {test_total}
Unique Users: {test_users}
Unique Items: {test_items}
""".format(
    train_total=len(train),
    train_users=len(train[USER_COL].unique()),
    train_items=len(train[ITEM_COL].unique()),
    test_total=len(test),
    test_users=len(test[USER_COL].unique()),
    test_items=len(test[ITEM_COL].unique()),
))


Train:
Total Ratings: 74992
Unique Users: 943
Unique Items: 1653

Test:
Total Ratings: 25008
Unique Users: 943
Unique Items: 1444



## 2 Train the SAR Model

### 2.1 Instantiate the SAR algorithm and set the index

We will use the single node implementation of SAR and specify the column names to match our dataset (timestamp is an optional column that is used and can be removed if your dataset does not contain it).

Other options are specified to control the behavior of the algorithm as described in the [deep dive notebook](../02_model_collaborative_filtering/sar_deep_dive.ipynb).

In [6]:
model = SAR(
    col_user=USER_COL,
    col_item=ITEM_COL,
    col_rating=RATING_COL,
    col_timestamp=TIMESTAMP_COL,
    similarity_type=SIMILARITY_TYPE, 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)

### 2.2 Train the SAR model on our training data, and get the top-k recommendations for our testing data

SAR first computes an item-to-item ***co-occurence matrix***. Co-occurence represents the number of times two items appear together for any given user. Once we have the co-occurence matrix, we compute an ***item similarity matrix*** by rescaling the cooccurences by a given metric (Jaccard similarity in this example). 

We also compute an ***affinity matrix*** to capture the strength of the relationship between each user and each item. Affinity is driven by different types (like *rating* or *viewing* a movie), and by the time of the event. 

Recommendations are achieved by multiplying the affinity matrix $A$ and the similarity matrix $S$. The result is a ***recommendation score matrix*** $R$. We compute the ***top-k*** results for each user in the `recommend_k_items` function seen below.

A full walkthrough of the SAR algorithm can be found [here](../02_model_collaborative_filtering/sar_deep_dive.ipynb).

In [7]:
with Timer() as train_time:
    model.fit(train)

print(f"Took {train_time.interval} seconds for training.")

INFO     Collecting user affinity matrix
INFO     Calculating time-decayed affinities
INFO     Creating index columns
INFO     Calculating normalization factors
INFO     Building user affinity sparse matrix
INFO     Calculating item co-occurrence
INFO     Calculating item similarity
INFO     Using jaccard based similarity
INFO     Done training


Took 0.5214087999775074 seconds for training.


In [8]:
with Timer() as test_time:
    top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

print("Took {} seconds for prediction.".format(test_time.interval))

INFO     Calculating recommendation scores
INFO     Removing seen items


Took 0.33369579998543486 seconds for prediction.


In [9]:
top_k.head()

,userID,itemID,prediction
0,1,433,2.910697
1,1,204,2.906224
2,1,403,2.906136
3,1,174,2.870639
4,1,70,2.863253


### 2.3. Evaluate how well SAR performs

We evaluate how well SAR performs for a few common ranking metrics provided in the `python_evaluation` module. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with SAR. User, item and rating column names are specified in each evaluation method.

In [10]:
# Ranking metrics
eval_map = map_at_k(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL, k=TOP_K)
eval_ndcg = ndcg_at_k(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL, k=TOP_K)
eval_precision = precision_at_k(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL, k=TOP_K)
eval_recall = recall_at_k(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL, k=TOP_K)


In [11]:
# Rating metrics
eval_rmse = rmse(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL)
eval_mae = mae(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL)
eval_rsquared = rsquared(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL)
eval_exp_var = exp_var(test, top_k, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL)


In [12]:
positivity_threshold = 2
test_bin = test.copy()
test_bin[RATING_COL] = binarize(test_bin[RATING_COL], positivity_threshold)

top_k_prob = top_k.copy()
top_k_prob[PREDICTION_COL] = minmax_scale(top_k_prob[PREDICTION_COL].astype(float))

eval_logloss = logloss(
    test_bin, top_k_prob, col_user=USER_COL, col_item=ITEM_COL, col_rating=RATING_COL
)


In [13]:
print("Model:\t\tSAR",
      "Top K:\t\t%d" % TOP_K,
      "MAP:\t\t%f" % eval_map,
      "NDCG:\t\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall,
      "RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "R2:\t\t%f" % eval_rsquared,
      "Exp var:\t%f" % eval_exp_var,
      "Logloss:\t%f" % eval_logloss,
      sep='\n')

Model:		SAR
Top K:		10
MAP:		0.106959
NDCG:		0.379533
Precision@K:	0.331071
Recall@K:	0.176837
RMSE:		1.229246
MAE:		1.033912
R2:		-0.511334
Exp var:	0.098494
Logloss:	0.569153


In [14]:
# Now let's look at the results for a specific user
user_id = 200

user_input = pd.DataFrame(dict(userID=[user_id]))

In [15]:
ground_truth = test[test[USER_COL] == user_id]
prediction = model.recommend_k_items(user_input, remove_seen=True)

df = pd.merge(ground_truth, prediction, on=[USER_COL, ITEM_COL], how="left").dropna().sort_values(by=PREDICTION_COL, ascending=False)
df

INFO     Calculating recommendation scores
INFO     Removing seen items


,userID,itemID,rating,timestamp,prediction
46,200,423,5.0,884129275,3.815873
34,200,96,5.0,884129409,3.733348
43,200,56,4.0,884128858,3.640387
0,200,234,4.0,884129381,3.610741



The prediction of SAR is not a rating, but a score that measures the affinity between a user and an item. The stronger the affinity, the higher the change that the user is interested in that item. Therefore, we are interested in the order of the prediction, not in the value of the prediction itself. 